## **PRODUÇÃO - A (LOAD)**

In [ ]:
# pandas para manipulação de dataframes
import pandas as pd

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

# pacotes de visualização de dados
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import seaborn as sns; sns.set()

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

# pacotes para estatísticas e criação de modelos
import numpy as np
from scipy import stats

# CLUSTER
from sklearn.cluster import KMeans

# PCA 
from sklearn.decomposition import PCA
# Import the KElbowVisualizer method 
from yellowbrick.cluster import KElbowVisualizer

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

# upload de arquivos 
from google.colab import files

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

!pip install  -q pytz
import pytz
from datetime import datetime, timezone
from pytz import all_timezones

'''
for tz in pytz.all_timezones:
    print (tz)
'''

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
# Configurando o número máximo de linhas a se mostrar
pd.set_option('display.max_row', 5000)

# Configurando o número máximo de colunas a se mostrar
pd.set_option('display.max_columns', 50)

# Aumentando o número de caracteres a serem exibidos numa coluna de texto
pd.options.display.max_colwidth = 500

# Desligando a notificação setcopywarning - Já contolamos a situação
pd.set_option('mode.chained_assignment', None)


!pip install -U -q PyDrive
import os
import datetime as DT
import io
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import googleapiclient.discovery
import numpy as np 
import requests
import json
import time
from pandas.io.json import json_normalize

# Set ipython's max row display
pd.set_option('display.max_row', 10000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 150)
pd.options.display.max_colwidth = 500

# ==============================================================================================================================
# ==============================================================================================================================
# ==============================================================================================================================

!pip install -U -q tweepy
!pip install -U -q textblob
!pip install -U -q TwitterAPI
from TwitterAPI import TwitterAPI, TwitterOAuth, TwitterRequestError, TwitterConnectionError, TwitterPager

import tweepy
from textblob import TextBlob

!pip install  -q nltk
!pip install  -q unidecode
import nltk
import re
from unidecode import unidecode
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('portuguese'))
print(stopwords)
!pip install  -q emojis
import emojis

'''
consumer_key = "XXXXXXXXXXXXXXXXXXXXX" 
consumer_secret = "XXXXXXXXXXXXXXXXXXXXX"
access_key = "XXXXXXXXXXXXXXXXXXXXX"
access_secret = "XXXXXXXXXXXXXXXXXXXXX"
'''

# Twitter authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)   
auth.set_access_token(access_key, access_secret) 
  
# Creating an API object 
api = tweepy.API(auth, wait_on_rate_limit=True)
api

api_v2 = TwitterAPI(consumer_key, consumer_secret, access_key, access_secret, api_version='2')

# ==============================================================================================================================
# ==============================================================================================================================
# ==============================================================================================================================

from google.colab import drive
drive.mount('/content/drive/')

## **PRODUÇÃO - B (RUN)**

In [ ]:
# ==============================================================================================================================
# ==============================================================================================================================
# ==================================== TWEETS DOS TIMES ATRAVÉS DO TWEEPY
# ==============================================================================================================================
# ==============================================================================================================================

df_schedule0 = pd.read_csv('/content/drive/My Drive/futebol_emocoes/times_do_brasileirao.csv', error_bad_lines=False, sep=',')
df_schedule0
df_schedule0.head(20)

df_schedule1 = df_schedule0[:2]
df_schedule1

tweet_list = []

for games in df_schedule0.itertuples():

  nome            = getattr(games, 'nome')
  perfil_twitter  = getattr(games, 'perfil_twitter') 
  dt              = getattr(games, 'dt')
  hr_ini          = getattr(games, 'hr_ini')
  hr_fim          = getattr(games, 'hr_fim')
  
  print(nome)
  #new_tweets = tweepy.Cursor(api.user_timeline, screen_name="Flamengo", tweet_mode='extended', since=1399102279203102721).items(100)
  new_tweets = tweepy.Cursor(api.user_timeline, screen_name=perfil_twitter, tweet_mode='extended', until=dt).items(100)


  for tweet_obj in new_tweets:
      #print(tweet_obj)

      #print(tweet_obj.user.screen_name)
      user_screen_name = tweet_obj.user.screen_name
      id = tweet_obj.id
      created_at = tweet_obj.created_at
      full_text = tweet_obj.full_text
      followers_count = tweet_obj.user.followers_count
      friends_count = tweet_obj.user.friends_count
      listed_count = tweet_obj.user.listed_count
      favorite_count = tweet_obj.favorite_count
      retweet_count = tweet_obj.retweet_count
      
      #HASHTAGS
      try:
          hashtags_list = tweet_obj['entities']['hashtags']
          hashtags = [hashtag['text'] for hashtag in hashtags_list]
          if len(hashtags_list)>1:
              temp['hashtags'] = ",".join(hashtags)
          else:
              hashtags = hashtags_list[0]['text']
      except:
              hashtags = ''

      #LOCALIZAÇÃO
      try:
          location = tweet_obj['user']['location']
      except:
          location = None
      
      #URL
      try:
          url = tweet_obj['entities']['urls'][0]['url']
      except:
          url = ''

      d = {'nome_time': nome, 'screen_name': perfil_twitter, 'dt_jogo': dt, 'hr_ini': hr_ini, 'hr_fim': hr_fim,
          'user_screen_name': user_screen_name, 'id': id, 'created_at': created_at, 'full_text': full_text, 'followers_count': followers_count, 
          'friends_count': friends_count, 'listed_count': listed_count, 'hashtags': hashtags, 'location': location, 'url': url, 
          'favorite_count': favorite_count, 'retweet_count': retweet_count }         
      #print(d)
      ooo = pd.DataFrame(d, index=[0])

      ooo['hora_brasil'] = pd.DatetimeIndex(pd.to_datetime(ooo['created_at'],unit='ms')).tz_localize('UTC').tz_convert('Brazil/East')
      ooo['dt'] = ooo['hora_brasil'].dt.date

      ooo = (ooo.set_index('hora_brasil').between_time(hr_ini, hr_fim).reset_index().reindex(columns=ooo.columns))

      url = "/content/drive/My Drive/futebol_emocoes/"
      ext   = '.csv'
      hifen = "_"
      final_url = f'{url}{nome}{hifen}{dt}{ext}'
      ooo.to_csv(final_url)

      tweet_list.append(ooo) 


df_game_plays = pd.concat(tweet_list)
df_game_plays.head(2000)
df_game_plays.info()
dia_de_jogo = df_game_plays
dia_de_jogo

# ==============================================================================================================================
# ==============================================================================================================================
# ==================================== EXTRAÇÃO DOS CONVERSATION ID´S - TWITTER API V2
# ==============================================================================================================================
# ==============================================================================================================================

conv_ids = []
 
for row in dia_de_jogo.itertuples():
        id          = getattr(row, 'id')
        nome_time   = getattr(row, 'nome_time')

        TWEET_ID = id
        TWEET_FIELDS = 'conversation_id'
        
        try:
            r = api_v2.request(f'tweets/:{TWEET_ID}', {'tweet.fields': TWEET_FIELDS})

            for item in r:

                  conversation_id   = item['conversation_id']
                  original_tweet_id = item['id']
                  #print(item)
                  #print(item['conversation_id'])
                  #print(item['id'])

                  d = {'nome_time': nome_time, 'original_tweet_id': original_tweet_id, 'conversation_id': conversation_id}         
                    
                  ooo = pd.DataFrame(d, index=[0])  
                  conv_ids.append(ooo)

        except TwitterRequestError as e:
            print(e.status_code)
            for msg in iter(e):
                print(msg)

        except TwitterConnectionError as e:
            print(e)

        except Exception as e:
            print(e)
        
df_conversation_ids = pd.concat(conv_ids)
df_conversation_ids
df_conversation_ids['conversation_id_num'] = df_conversation_ids['conversation_id'].astype(int)
df_conversation_ids

# ==============================================================================================================================
# ==============================================================================================================================
# ==================================== EXTRAÇÃO DAS RESPOSTAS - TWITTER API V2
# ==============================================================================================================================
# ==============================================================================================================================


replies = []
user=[]
for row in df_conversation_ids.itertuples():
   conversation_id_num = getattr(row, 'conversation_id_num')
   original_tweet_id   = getattr(row, 'original_tweet_id')
   nome_time           = getattr(row, 'nome_time')   
   #print(conversation_id_num)

   headers = {'Authorization': "Bearer " + str('YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY') }
   url = "https://api.twitter.com/2/tweets/search/recent?query="
   conversation = "conversation_id:"
   tweet_fieds = "&tweet.fields=id,text,source,created_at,author_id"
   user = "&expansions=author_id,geo.place_id&user.fields=id,name,username,created_at,description,public_metrics"
   place = "&place.fields=country,full_name,geo,name,place_type"    
   max_results=";max_results=100"
   #final_url = f'{url}{conversation}{conversation_id_num}{tweet_fieds}{place}{max_results}'
   final_url = f'{url}{conversation}{conversation_id_num}{tweet_fieds}{user}{place}{max_results}'
   #print(final_url)
   try:
        response = requests.get( final_url, headers=headers)
        response_data = (response.json())
        
        #print(response_data)
        for n in range(len(response_data['data'])):
            #print(n)
            tweet                = response_data['data'][n]['text']
            original_tweet_id    = original_tweet_id
            conversation_id      = conversation_id_num
            created_at           = response_data['data'][n]['created_at']                        
            reply_user_id        = response_data['data'][n]['author_id']
            reply_source         = response_data['data'][n]['source']
          
            if "geo" in response_data['data'][n]:
                      geo_id            = response_data['data'][n]['geo']['place_id']
                      #print(response_data['data'][n]['geo']['place_id'])
            else: 
                      geo_id            = np.nan  
          
            d = {'nome_time': nome_time, 'original_tweet_id': original_tweet_id, 'conversation_id': conversation_id, 'tweet_text': tweet, 
                'created_at': created_at, 'reply_user_id': reply_user_id, 'geo_id': geo_id, 'reply_source': reply_source}     
            
            #print(nome_time,conversation_id) 
            ooo = pd.DataFrame(d, index=[0])   
            replies.append(ooo) 
        time.sleep(2)
   except:
        pass
df_replies0 = pd.concat(replies)
df_replies0

df_replies0.to_csv('/content/drive/My Drive/futebol_emocoes/rodada_1.csv')

# ==============================================================================================================================
# ==============================================================================================================================
# ==================================== PRÉ TRATAMENTO DOS TWEETS 
# ==============================================================================================================================
# ==============================================================================================================================

# cópia de segurança
df_replies0_bkp = df_replies0
# df_replies0 = df_replies0_bkp

#remove números e caracteres especiais 
df_replies0['tweet_text'] = df_replies0['tweet_text'].apply(lambda x: re.sub('[0-9]!"#$%&\(\)*\+,-\./:;<=>?@\\\^_`{\|}~', ' ', x))
df_replies0

# CONVERTE TODAS AS LETRAS PARA MINÚSCULO

df_replies0['tweet_text'] = df_replies0['tweet_text'].apply(lambda x: x.lower())

# EXTRAINDO HASHTAGS

df_replies0['hashtag'] = df_replies0['tweet_text'].apply(lambda x: re.findall(r"#(\w+)", x))

# EXTRAINDO MENÇÕES

df_replies0['mentions'] = df_replies0['tweet_text'].apply(lambda x: re.findall(r"@(\w+)", x))

# EXTRAINDO EMOJIS

df_replies0['emojis']  = df_replies0['tweet_text'].apply(lambda x: emojis.get(x))

#
# REMOVING LINKS
#

df_replies0['tweet_semlinks'] = df_replies0['tweet_text'].apply(lambda x: re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", x))
df_replies0.head(2000)

#
# REMOVING MENTIONS
#

df_replies0['tweet_semmencoes'] = df_replies0['tweet_semlinks'].apply(lambda x: re.sub('@[\w]+',"", x))
df_replies0

#
# REMOVING EMOJIS
#

df_replies0['tweet_sememojis'] = df_replies0['tweet_semmencoes'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
df_replies0

stopwords = ["a","agora","ainda","alguém","algum","alguma","algumas","alguns","ampla","amplas","amplo","amplos","ante","antes","ao","aos","após","aquela","aquelas","aquele","aqueles",
"aquilo","as","até","através","cada","coisa","coisas","com","como","contra","contudo","da","daquele","daqueles","das","de","dela","delas","dele","deles","depois","dessa","dessas",
"desse","desses","desta","destas","deste","deste","destes","deve","devem","devendo","dever","deverá","deverão","deveria","deveriam","devia","deviam","disse","disso","disto","dito",
"diz","dizem","do","dos","e","é","ela","elas","ele","eles","em","enquanto","entre","era","essa","essas","esse","esses","esta","está","estamos","estão","estas","estava","estavam",
"estávamos","este","estes","estou","eu","fazendo","fazer","feita","feitas","feito","feitos","foi","for","foram","fosse","fossem","grande","grandes","há","isso","isto","já","la","lá",
"lhe","lhes","lo","mas","me","mesma","mesmas","mesmo","mesmos","meu","meus","minha","minhas","muita","muitas","muito","muitos","na","não","nas","nem","nenhum","nessa","nessas","nesta",
"nestas","ninguém","no","nos","nós","nossa","nossas","nosso","nossos","num","numa","nunca","o","os","ou","outra","outras","outro","outros","para","pela","pelas","pelo","pelos","pequena",
"pequenas","pequeno","pequenos","per","perante","pode","pude","podendo","poder","poderia","poderiam","podia","podiam","pois","por","porém","porque","posso","pouca","poucas","pouco",
"poucos","primeiro","primeiros","própria","próprias","próprio","próprios","quais","qual","quando","quanto","quantos","que","quem","são","se","seja","sejam","sem","sempre","sendo",
"será","serão","seu","seus","si","sido","só","sob","sobre","sua","suas","talvez","também","tampouco","te","tem","tendo","tenha","ter","teu","teus","ti","tido","tinha","tinham","toda",
"todas","todavia","todo","todos","tu","tua","tuas","tudo","última","últimas","último","últimos","um","uma","umas","uns","vendo","ver","vez","vindo","vir","vos","vós","q","pro","pra","vem",
"pq","você","deu","hj","ta","cep","1kg","necessidade","não","nao","nesse","aqui","time","mais","tá","aí","vocês","voces","série","serie","faz","ser","cara","fez","dá",
"dão", "dando", "dar", "demaiisssss", "demais", "demaisss", "demaissss", "demaissssss", "desda", "desde", "dia", "dias", "dois", "doisq", "duas", "eh", "ehh", "ehhhh", "então", "fala", "falado",
"falador", "faladoum", "falam", "falamos", "falando", "falar", "falaram", "falarem", "falasse", "falastrão", "falava", "falavam", "falcão", "fale", "falei", "falou", "foto", "fotos", "hoje",
"hojese", "hojetime", "hojr", "i", "ia", "iae", "ir", "ja", "joga", "jogávamos", "jogão", "jogaa", "jogaaar", "jogada", "jogadaça", "jogadas", "jogadasfaz", "jogadassaa", "jogadassas", "jogado", 
"jogador", "jogadorão", "jogadoras", "jogadores", "jogadoresmais", "jogadorzinho", "jogaf", "jogai", "jogam", "jogamos", "jogando", "jogar", "jogará", "jogaram", "jogarem", "jogarmonos", "jogarrrrr", 
"jogasso", "jogava", "jogndo", "jogo", "jogono", "jogopara", "jogopelo", "jogos", "jogou", "jogue", "joguei", "joguem", "joguin", "joguinho", "joguinhos", "junta", "juntar", "junto", 
"juntos", "juntoseshaylonnow", "juntosnobrasileirão", "mim", "msg", "msm", "mt", "mta", "mto", "mts", "mttt", "nada", "nadaÃ³timo", "nadaaaaa", "número", "números", "naquela", "naquele", "nasa",
"nations", "nba", "nd", "ndd", "ne", "nela", "nelas", "nele", "neles", "nenhuma", "nenhumafica", "nenhuna", "nesses", "neste", "new", "ngm", "ninguem", "ninhum", "nisso", "nois", "noite",
"noitee", "nome", "nomes", "numero", "numeros", "oh", "ohhh", "oi", "ok", "okay", "okkkkk", "ola", "olá", "olaf", "onde", "ops", "optou", "oq", "oqkkkkk", "oquanto", "oque", "p", "pix", 
"qd", "qdo", "qlq", "qm", "qnd", "qnts", "qq", "qqr", "qtos", "quão", "qué", "qualquer", "quanta", "quantas", "quer", "queríamos", "querem", "querer", "queres", "queria", "quero", "quis", 
"r", "ready", "rt", "semana", "semanal", "semanas", "silva", "sim", "simm", "simmm", "simmmm", "simmmmm", "sla", "slc", "slk", "small", "socorro", "sofri", "sofria", "sofrido", "sofridos", 
"sofrimento", "star",  "station", "stats", "still", "stop", "story", "tá", "tão", "tal", "tanta", "tantão", "tantas", "tanto", "tantooo", "tantos", "tao", "tava", "tavam", "tb", "tbem", "tbm", 
"tchê", "tchapordeus", "tchau", "tche", "td", "tdas", "tds", "than", "thanks", "that", "thats", "the", "their", "them", "then", "there", "they", "theyll", "third", "this", "thisgo", "thyere", 
"tik", "tiktok", "till", "tilt", "timing", "tl", "tlg", "tlgd", "tlvz", "tm", "tmb", "tmj", "tml", "to", "três", "trás", "tuddoooo", "tuddooooooooo", "tudooo", "tudoooo", "tudooooo", "tudoooooo",
"tudooooooo", "tudoooooooo", "tudooooooooo", "tudooooooooooooo", "tudooooooooooooooo", "tudoooooooooooooooooooo", "tudoooooooooooooooooooooooooooooooooooo", "tuuudoooooo", "tuuuudo", 
"tuuuudoooooooo", "tuuuuuudoooooo", "tuuuuuudooooooo", "tuuuuuuuddddddooooooooo", "tuuuuuuuudddddddooooooooooo", "tuuuuuuuudoooooo", "tuuuuuuuuudo", "tuuuuuuuuuuuuuuuuuuudo", "tv", "tweet", "tweeta",
"tweetar",  "tweets", "twit", "twitado", "twitar", "twitter", "vc", "vco", "vcs", "vdd", "very", "vi", "vlw", "voce", "wait", "wallpaper", "want", "wat", "way", "what", "whats", "whatsapp", "where", 
"win", "winners", "wins", "with", "woke", "wont", "worth", "year", "years", "yes", "yesssssss", "you", "your", "youth", "youtube", "youtuber", "youu", "zap", "dnv", "abel", "adriano", "alberto", 
"alex", "alexandrepato", "alexi", "alfredo", "alisson", "allyson", "allysoneventos", "alves", "alvessilvinho", "alyson", "alysson", "andré", "anual", "anuidade", "anula", "anulaaaaaa", "anulado", 
"anulando", "anular", "anule", "anulou", "arboleda", "arbolenda", "arejuela", "bandeira", "bandeirinha", "bruno", "brupedro", "bueno", "camacho", "cortez", "crespo", "cuca", "cucabol", "danilo",
"davi", "david", "diego", "douglas", "douglascosta", "edenilso", "edenilsom", "edenilson", "ednilson", "felipão", "felipao", "felipe", "felipee", "felipemello", "felipemelonotimao", "fernandes", 
"fernandinho", "fernando", "fernandoasafil", "ferreira", "ferreirinha", "ferrer", "ferrerinha", "fred", "gabgol", "gabi", "gabigol", "gabinete", "gabriel", "galhardo", "galhardooooo", "galhargol", 
"galiotte", "ganso", "gerson", "gil", "gustavo", "gustavooo", "henriqu", "henrique", "hernández", "hernan", "hernandez", "hernane", "hernanes", "hernanesnÃ£o", "jean", "leo", "luan", "lucas",
"luis", "luiz", "luizao", "mancini", "marcão", "marcelo", "marco", "marcos", "maria", "mariano", "marinho", "mario", "mariza", "marlon", "marques", "marquinhos", "marrony", "marti", "martinelli",
"martorelli", "mateus", "mateusinho", "mateusinhoq",  "mateusvital", "matheu", "matheus", "matheusinho", "matheuzinho", "maucini", "maurício", "mauricio", "maxuell", "maxwell", "maycon", 
"mayke", "maykeo", "mazeti", "mazzeti", "mazzetti", "mello", "melo", "moisés", "neiyton", "nenê", "nene", "neneeee", "neneeeee", "neneeeeeeeeeeeeee", "nenene", "nunes", "orejeula", "orejuela",
"orejuelaaaaaaaaaaaaaaaaaa", "orejuelaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa", "orejuella", "pablo", "pabloe", "pacheco", "pachecoooo", "pachecoooooo", "paolo", "pardabel", "pardal", "pato", "patric", 
"patrick", "patrik", "paulina", "paulinho", "paulino", "paulinos", "paulo", "pauloooo", "paulouilian", "paulovictornão",  "pedrinho", "pedro", "pedrocoisa", "pedroe", "pedroebeca", "pedrokkkk", 
"pepe", "picachu", "pikachu", "pikachuuuuu", "pikachuzado", "pyerre", "pyerreeee", "pyerreeeeeee", "rafael", "rafel", "rafinha", "ramí­rez", "ramires", "ramirez", "ramiro", "ramirojogador", "ramon", 
"ramos", "ramosgtedenilson", "raul", "ravanelli", "renato", "renatogauchonaselecao", "retardada", "retardado", "ribamar", "ribas", "ribeiro", "ricardin", "ricardinho", "ricardinhooooooooo", "ricardo",
"ricargol", "richard", "richarlison", "robergol", "robershow", "roberson", "robert", "roberto", "roberts", "robertson", "roberty", "robertzzzzz", "robinho", "roblox", "robson", "rodi", 
"rodibala", "rodiii", "rodilindis", "rodilindo", "rodilindooo", "rodilindoooo", "rodilindooooo", "rodilindooooooooo", "rodilixo", "rodinei", "rodlindo", "rodnei", "rodolfo", "rodolfopara", "rodrigo",
"rodriguéz", "rodrigues", "rodriguez", "rodriguinho", "rogério", "rogenio", "roger", "romário", "romarinho", "romario", "romero", "romildastico", "romildinho", "romildo", "ronaldao", 
"ronaldinho", "ronaldinhogaucho", "ronaldo", "roni", "ronielson", "ronny", "rony", "rooney", "rossi", "rossicley", "shaylon", "sylvinho", "sylvio", "taison", "talisson", "talles", "talleshernanes", 
"teixeira", "teixeiras", "thiago", "thiagol", "thiaguinho", "tinga", "valentim", "valentimm", "valentin", "vandeco", "vanderlei", "vanderson", "victor", "victoria", "victoro", "vital", "vitinho",
"vito", "vitor", "vitu", "walter", "waltermas", "wanderson", "wel", "welington", "well", "wellington", "wellinton", "wendel", "wescley", "wesley", "weverton", "wiliam", "wililam", "will", "william",
"willian", "willliam", "xavi", "xavier", "yago", "yashin", "ytalo", "yuri", "zé", "ze", "zico", "jhonata", "jhonatan", "jhonattan", "jhonnatan", "jhonny", "jhonota", "joão", "jóia", "joao",
"lima", "muricy", "prass", "prasss"]

#
# REMOVING STOPWORDS
# 

#https://stackoverflow.com/questions/54366913/removing-stopwords-from-a-pandas-dataframe

df_replies0['tweet_div'] = df_replies0.tweet_semmencoes.str.replace("[^\w\s]", "").str.lower()
df_replies0['tweet_semstop'] = df_replies0['tweet_div'].apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords]))
df_replies0.head(2000)

# apagando as colunas desnecessárias

del df_replies0['tweet_div']
del df_replies0['tweet_semlinks']
del df_replies0['tweet_semmencoes']
del df_replies0['tweet_sememojis']

df_replies0.info()

#df_replies0.dropna(subset=['tweet_semstop'], inplace=True)

df_replies0.head(2)

# ==============================================================================================================================
# ==============================================================================================================================
# ==================================== WORDCLOUDS
# ==============================================================================================================================
# ==============================================================================================================================

import matplotlib.pyplot as plt   # for wordclouds & charts
import seaborn as sns   # for charts
sns.set_style("whitegrid");   # chart background style
plt.rcParams['figure.dpi'] = 360   # for high res chart output
from wordcloud import WordCloud   # for the wordcloud :)

df_replies1 = df_replies0.nome_time.unique()
df_replies1_5 = pd.DataFrame(df_replies1, columns=['nome_time'])

# ==================================== CARREGANDO O ARQUIVO COM OS COLORMAPS ESPECÍFICOS DE CADA TME

df_times_cores = pd.read_csv('/content/drive/My Drive/futebol_emocoes/times_do_brasileirao-cores.csv', error_bad_lines=False, sep=';')
df_times_cores

df_replies2 = pd.merge(df_replies1_5, df_times_cores, left_on=['nome_time'], right_on=['nome'], how='left')
df_replies2

appended_data = []

for teams in df_replies2.itertuples():
    nome                = getattr(teams, 'nome_time')
    colormap            = getattr(teams, 'colormap')
    li = list(nome.split(","))
    #nome = ['Cuiaba']
    df_replies3= df_replies0[ (df_replies0.nome_time.isin(li)) ]
    df_replies3
  
    df_freq = df_replies3.tweet_semstop.str.split(expand=True).stack().value_counts()

    url = "/content/drive/My Drive/futebol_emocoes/wordclouds/"
    ext   = '.png'
    ext2   = '.csv'
    hifen = "_"
    freq = "freq"
    rodada1 = "rodada1"
    final_url = f'{url}{nome}{hifen}{rodada1}{ext}'

    tweet_wordcloud = WordCloud(background_color="white", width=380, height=380, colormap=colormap, max_words=200 ).generate_from_frequencies(df_freq)

    tweet_wordcloud.to_file(final_url)

    type(df_freq)
    df_freq2 = pd.DataFrame(df_freq)
    df_freq2['nome'] = getattr(teams, 'nome_time')
    final_freq_url = f'{url}{nome}{hifen}{freq}{hifen}{rodada1}{ext2}'
    
    df_freq3 = df_freq2.reset_index()
    df_freq3
    df_freq3.columns.values[0] = "palavra"
    df_freq3.columns.values[1] = "freq"
    df_freq3.columns.values[2] = "time"

    df_freq3.to_csv(final_freq_url)
    
    #print(d)
    ooo = pd.DataFrame(df_freq3)
    #print(ooo)
    appended_data.append(ooo)

df_words = pd.concat(appended_data)
df_words

df_words.to_csv('/content/drive/My Drive/futebol_emocoes/freq_palavras.csv')